In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf
import csv
import sys
sys.path.append('/storage/home/hcoda1/4/ashelby8/Manitoba/pyNab/src/')
import nabPy as Nab
sys.path.append('/storage/home/hcoda1/4/ashelby8/calibration/calibration/')
from FitClass import SnCalibration
from config import conf
import FitFuncs
import pandas as pd
import pylab as py
np.set_printoptions(threshold=sys.maxsize)

In [2]:
pd.set_option("display.max_colwidth", 15)

In [72]:
with open('/storage/home/hcoda1/4/ashelby8/output1304.csv') as f:
    csv.DictReader(f)
    data = pd.read_csv(f)

In [73]:
data

,run,pixel,Bias Voltage,Proton,Proton Energy,Cd109,Sn113,Date Time [UTC],trap rise,trap length,trap decay,ecap,chi2_e,xray,chi2_x,CE,Xray,CE hist,Xray hist
0,1304,76,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[67.309267...,[np.float64...,[ 3.0778278...,3104.077276,"[1, 2]",3,"[[24, 16, 1...",[1716 1633 ...
1,1304,77,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[59.572546...,[np.float64...,[ 4.6938587...,885.596444,"[1, 2]",3,"[[17, 13, 1...",[1773 1441 ...
2,1304,78,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[38.765647...,[np.float64...,[ 5.4012051...,1128.700105,"[1, 2]",3,"[[16, 16, 1...",[555 508 51...
3,1304,87,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[32.717400...,[np.float64...,[ 4.2950038...,5655.672552,"[1, 2]",3,"[[13, 15, 1...",[2155 2138 ...
4,1304,88,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[49.189179...,[np.float64...,[ 4.8919238...,3901.020369,"[1, 2]",3,"[[16, 12, 1...",[2429 2296 ...
5,1304,96,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[20.865379...,[np.float64...,[ 2.6037786...,431.124599,"[1, 2]",3,"[[12, 9, 9,...",[1090 863 ...
6,1304,97,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[35.864973...,[np.float64...,[ 5.3641047...,2352.013772,"[1, 2]",3,"[[16, 19, 1...",[2167 1871 ...
7,1304,98,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[37.574231...,[np.float64...,[ 3.6153483...,3346.216808,"[1, 2]",3,"[[7, 13, 15...",[1858 1727 ...
8,1304,101,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[10.783606...,[np.float64...,[ 1.1959726...,107.330347,"[1, 2]",3,"[[7, 11, 12...",[361 244 15...
9,1304,103,-240,False,0,True,False,2021-11-20 ...,1250,50,1250,[[24.601137...,[np.float64...,0,0.000000,"[1, 2]",3,"[[3, 7, 8, ...",0


In [74]:
data['CE hist'][0].replace('[','').replace(']','').replace('(','').replace(')','').split('array')[1]


IndexError: list index out of range

In [142]:
pd.Series(data['chi2_e'][0],dtype=np.float64)

ValueError: could not convert string to float: '[np.float64(39.158641189429545), np.float64(29.306732018928827)]'

In [119]:
data['CE'][0]

'[1, 2]'

In [ ]:
nrows,ncols=2,2
py.figure(figsize=(4*ncols,3*nrows))


x = np.arange(500,800)

#initializes the electron capture peak fitting
conf['xray'] = 'OFF'

for i in range(len(data)):
    if data['CE'][i]==3:
        
        #need to initialize the class to get the correct fit function based on the same recorded initilizing flag in column "CE"
        conf['capture'] = 'three'
        Sn = SnCalibration()
        
        #plot all the fits with the same initilization on the same graph
        ax=py.subplot(nrows,ncols,1)
        
        #need to convert the stored parameter string into the corresponding numerical values and plot Sn.get_fit (the fitting function)
        ax.plot(x,Sn.get_fit(x,*np.float64(data['ecap'][i].replace('[','').replace(']','').split())),label = '%d'%i)
        ax.legend()
        
    if data['CE'][i]==2:
        
        conf['capture'] = 'two'
        Sn = SnCalibration()
        
        ax=py.subplot(nrows,ncols,2)
        
        ax.plot(x,Sn.get_fit(x,*np.float64(data['ecap'][i].replace('[','').replace(']','').split())),label = '%d'%i)
        ax.legend()
        
    if data['CE'][i]==1:
        
        conf['capture'] = 'one'
        Sn = SnCalibration()
        
        ax=py.subplot(nrows,ncols,3)
        ax.plot(x,Sn.get_fit(x,*np.float64(data['ecap'][i].replace('[','').replace(']','').split())),label = '%d'%i)
        
        ax.legend()
        
    if data['CE'][i]==0:
        print('not enough counts for fit: pixel %d'%i)
        conf['capture'] = 'zero'
#         Sn = SnCalibration()
        
#         ax=py.subplot(nrows,ncols,4)
#         ax.plot(x,Sn.get_fit(x,*np.float64(data['ecap'][i].replace('[','').replace(']','').split())),label = '%d'%i)
        
#         ax.legend()

py.tight_layout()